In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler


# pvlib imports
import pvlib
from pvlib.location import Location
from pvlib.pvsystem import PVSystem
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib import irradiance

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 14})

import re
import copy

In [2]:
# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
repo_name = 'net-load-forecasting'
assert os.getcwd()[-len(repo_name):] == "net-load-forecasting", "Working directory is not the git repo root directory"

Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\net-load-forecasting


In [3]:
clean_data_path = os.path.join(os.getcwd(),'data','clean_data')

In [8]:
pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key='15min/netload')

component,net_community_load
time,
2014-08-27 00:00:00,2973.965877
2014-08-27 00:15:00,2678.502833
2014-08-27 00:30:00,2776.943558
2014-08-27 00:45:00,2975.639441
2014-08-27 01:00:00,3332.794556
...,...
2016-12-29 22:45:00,4587.582423
2016-12-29 23:00:00,4447.781363
2016-12-29 23:15:00,3685.346414


In [ ]:
def get_accuracies_per_month(df, model_type, metrics:list = None):

    accuracy_dictionary = {}

    start_date = df.index[0]
    test_begin = df.index[-1] - pd.Timedelta(days = 30)
    for month in range(1,12):
        train_end = start_date + pd.Timedelta(weeks = 4*month)

        assert train_end < test_begin, "Reduce the number of months, so a test period remains."
        train = df[start_date:train_end]

        test = df[test_begin:]


        scaler_ml_features = MinMaxScaler()
        scaler_ml_target = MinMaxScaler()

        X_train, y_train = scaler_ml_features.fit_transform(train.iloc[:,1:]), scaler_ml_target.fit_transform(train.iloc[:,:1])
        X_test, y_test = scaler_ml_features.transform(test.iloc[:,1:]), scaler_ml_target.transform(test.iloc[:,:1])


        model = model_type()
        model.fit(X = X_train, y = y_train)
        predictions = model.predict(X_test)

        gt_unscaled = scaler_ml_target.inverse_transform(y_test.reshape(-1,1))
        predictions_unscaled = scaler_ml_target.inverse_transform(predictions.reshape(-1,1))

        df_y_test = pd.DataFrame({"ground_truth":gt_unscaled.squeeze(), "predictions": predictions_unscaled.squeeze()}, index = test.index)

        rmse = mean_squared_error(y_true=df_y_test["ground_truth"], y_pred= df_y_test["predictions"], squared=False) / test.max()[0]
        r2 = r2_score(y_true=df_y_test["ground_truth"], y_pred= df_y_test["predictions"])

        accuracy_dictionary[month] = rmse, r2

    df_accuracy = pd.DataFrame(accuracy_dictionary, index = ["rmse", "r2"]).T

    return df_accuracy, df_y_test



def physical_profile(row, df_irr):
    idx, latitude, longitude, tilt, azimuth, capacity = row

    temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS["sapm"][
        "open_rack_glass_glass"
    ]

    location = Location(latitude=latitude, longitude=longitude)

    pvwatts_system = PVSystem(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        module_parameters={"pdc0": capacity, "gamma_pdc": -0.004},
        inverter_parameters={"pdc0": capacity},
        temperature_model_parameters=temperature_model_parameters,
    )

    mc = ModelChain(
        pvwatts_system, location, aoi_model="physical", spectral_model="no_loss" #these are my model chain assumptions
    )
    mc.run_model(df_irr)
    results = mc.results.ac

    df_results = pd.Series(results)
    df_results.index = df_results.index.tz_localize(None)
    df_results.index.name = "timestamp"
    df_results.name = str(tilt) + ";" + str(azimuth)
    df_results.columns = [str(idx) + "_physical_profile"]

    return df_results


def pv_day_filter(data, lat, lon, tilt, azimuth, timesteplen):

    site = Location(lat, lon)
    index = data.index
    times = pd.date_range(index[0], index[-1], freq=str(timesteplen) + "T")
    clearsky = site.get_clearsky(times)
    solar_position = site.get_solarposition(times=times)
    # Use the get_total_irradiance function to transpose the GHI to POA
    POA_irradiance = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        dni=clearsky["dni"],
        ghi=clearsky["ghi"],
        dhi=clearsky["dhi"],
        solar_zenith=solar_position["apparent_zenith"],
        solar_azimuth=solar_position["azimuth"],
    )

    day_index = POA_irradiance[POA_irradiance["poa_global"] > 0].index

    data_day_values = data.reindex(day_index).dropna()

    return data_day_values



def get_df_compare_physical(df_meta, df_power, df_irr, test_period_index):

    """
    
    
    df_irr must have columns ['ghi', 'dni', 'dhi']

    """

    df_compares = []
    timesteplen = int(test_period_index.freq.delta.seconds/60) # in minutes
    
    for system_id in df_meta.index:
        df_meta_data_id = df_meta.loc[system_id]
        idx, lat, lon, tilt, azimuth, _ = df_meta_data_id.values
        
        df_irr = df_irr.reindex(test_period_index).dropna()
        df_physical_profile = physical_profile(df_meta_data_id, df_irr).to_frame(idx).resample("15T").mean().fillna(method = "bfill", limit = 4) 
        df_physical_profile = pv_day_filter(df_physical_profile, lat, lon, tilt, azimuth, timesteplen)

    
        df_power_id = df_power[[str(system_id)]].resample(str(timesteplen) + "T").mean()

        df_compare = pd.merge(df_power_id, df_physical_profile, left_index=True, right_index=True, how = "right").fillna(method = "bfill", limit= 4).dropna()
        df_physical_profile = pv_day_filter(df_compare, lat, lon, tilt, azimuth, timesteplen)
        df_compares.append(df_compare.reindex(test_period_index).dropna())

    return df_compares


def calc_accuracies_from_df_compares(df_compares:list, metrics:list, lat, lon, tilt, azimuth, timesteplen):

    """
    Inputs are a list of pd.DataFrames = [[df_predictions, df_ground_truth],[df_predictions, df_ground_truth], .... ] and a list of error metrics from sklearn.metrics

    Note that the mse will be reinterpreted as the nRMSE. This is because of the specific problem that has been analysis, i.e., the different capacities of pv systems. 

    Outputs are the error scores for each df in the dfs list for each error metric in a table that looks like:
    
    """


    metrics_names = [metric.__name__ for metric in metrics]
    df_acc = pd.DataFrame(columns = metrics_names, index = range(len(df_compares)))


    system_ids = []
    for i, df_compare in enumerate(df_compares):


        df_compare = pv_day_filter(df_compare, lat, lon, tilt, azimuth, timesteplen)
        system_ids.append(int(df_compare.columns[0]))

        for metric in metrics:
            metric_name = metric.__name__
            score = metric(df_compare.iloc[:,:1],df_compare.iloc[:,1:])
            if metric_name == "mean_squared_error":
                score = np.sqrt(score)
        
            df_acc.loc[i, metric_name] = score


    #transforming the mse into the rmse
    if "mean_squared_error" in metrics_names:

        df_acc["root_mean_squared_error"] = np.sqrt(df_acc["mean_squared_error"].astype("float32"))

        df_acc.drop(["mean_squared_error"], axis = 1, inplace=True)

    df_acc.index = system_ids

    return df_acc



def realistic_case_round(value:int, interval:int):

    """
    Rounding the tilt to interval'th degrees.
    """
    rounded_value = round(value / interval) * interval
    
    return rounded_value



def make_df_plot_from_stats(stats_d):
    df_plot = pd.DataFrame(stats_d).T.round(4)
    df_plot.columns = ["mean", "std"]
    return df_plot


def metric_skill_score(df_accs_baseline:pd.DataFrame, df_accs_model:pd.DataFrame, scenario_name:str):
    """
    Calculated as follows
    1 - (score_model / score_reference) and then we average over all systems
    """

    
    df_skill_scores = (1 - df_accs_model.div(df_accs_baseline))
    mean = df_skill_scores.mean().to_frame("expected value_" + scenario_name)

    std = df_skill_scores.std().to_frame("standard deviation_" + scenario_name)

    out = pd.concat([mean, std], axis = 1)

    return out




def random_angle_init(
                        df_meta:pd.DataFrame, 
                        df_power:pd.DataFrame,
                        df_irr:pd.DataFrame,
                        df_acc_baseline:pd.DataFrame,
                        test_period_index: pd.date_range, 
                        lat, lon, tilt, azimuth, timesteplen,
                        error_metric_list:list,
                        tilt_bounds:tuple = (10,60), 
                        azimuth_bounds:tuple = (0,360),
                        scenario_name = "P3"
                        ):
    """
    Use for scenarios in which angles are guessed by the aggregator. They are drawn *iters*-times from a uniform distribution for all systems within the optionally specified bounds.
    Note: df_power and df_irr need to be in the global namespace of the file. 
    """

    df_meta_random = copy.deepcopy(df_meta)
    random_tilts = np.random.randint(tilt_bounds[0], tilt_bounds[1], (df_meta_random.shape[0]))
    random_azimuths = np.random.randint(azimuth_bounds[0], azimuth_bounds[1], (df_meta_random.shape[0]))
    df_meta_random["tilt"] = random_tilts
    df_meta_random["azimuth"] = random_azimuths
    df_meta_compare = get_df_compare_physical(df_meta_random, df_power, df_irr, test_period_index= test_period_index)
    df_meta_accs = calc_accuracies_from_df_compares(df_meta_compare, error_metric_list,lat, lon, tilt, azimuth, timesteplen)
    df_skill_score = metric_skill_score(df_acc_baseline, df_meta_accs, scenario_name)
    
    return df_skill_score


def drop_duplicate_index(df, axis=0):
    """
    Removes all duplicate columns or index items of a pd.DataFrame. (Keeps first)
    """
    if axis == 0:
        df = df.loc[~df.columns.duplicated(keep="last"),:]
    elif axis == 1:
        df = df.loc[:,~df.columns.duplicated(keep="last")]
    else:
        raise ValueError("Make sure axis is either 0 (index) or 1 (column)")

    return df


def acc_bar_plot(df_plot: pd.DataFrame):
    today = datetime.datetime.today()
    for i in range(df_plot.shape[0]):
        means = df_plot.iloc[i].filter(like = "exp").sort_index()
        stds = df_plot.iloc[i].filter(like = "dev").sort_index()
        
        # finding the scenario names for the x axis labels
        idx = means.index
        result = [re.findall(r'_(.*)', s)[0] for s in idx]
        
        
        fig, ax = plt.subplots(figsize = (12,8))
        ax.set_axisbelow(True)
        ax.grid(axis = "y")
        ax.bar(x = result, height = means, yerr =stds, capsize = 10)
        ax.set_ylim(0,1.5*means.max())
        acc_score = means.name.replace("_", " ").title() + " Skill-Score"
        ax.set_ylabel(acc_score)
        fig.savefig(f"../../../Results/Plots/{acc_score}_{today}_persistence_ref.pdf")


def euro_cost_metric(gt, prediction, prices):
    pass



def get_df_compare_physical_new(df_meta, df_power, df_irr, test_period_index):

    """
    
    
    df_irr must have columns ['ghi', 'dni', 'dhi']

    """

    list_df_physical = []
    timesteplen = int(test_period_index.freq.delta.seconds/60) # in minutes
    
    for system_id in df_meta.index:
        df_meta_data_id = df_meta.loc[system_id]
        idx, lat, lon, tilt, azimuth, _ = df_meta_data_id.values
        
        df_irr = df_irr.reindex(test_period_index).dropna()
        df_physical_profile = physical_profile(df_meta_data_id, df_irr).to_frame(idx).resample("15T").mean().fillna(method = "bfill", limit = 4) 
        df_physical_profile = pv_day_filter(df_physical_profile, lat, lon, tilt, azimuth, timesteplen)

        list_df_physical.append(df_physical_profile)

    
    df_power_ = df_power.resample(str(timesteplen) + "T").mean()
    df_physical = pd.concat(list_df_physical, axis = 1).sum(axis = 1).to_frame("power_physical_model")
    df_compare = pd.merge(df_power_, df_physical, left_index=True, right_index=True, how = "left")
    df_compare = df_compare.fillna(0)

    return df_compare